# Data Preprocessing of Text Datasets

## 1. Diary-style Text Datasets 

Datasets:

1. [journal-entries-with-labelled-emotions](https://www.kaggle.com/datasets/madhavmalhotra/journal-entries-with-labelled-emotions) — 1500 journal entries labelled with 18 emotions from people reflecting on their day.
2. [Diary-Entry-To-Rap](https://huggingface.co/datasets/chaudha7/Diary-Entry-To-Rap) — 175 diary entries with age and gender writer's info plus transformed diary into rap text with certain mood and style.

From both datasets only texts will be taken.

### Read only diary-style text entries

In [6]:
import pandas as pd


journals_ds_dir = 'data/journals.csv'

journals_df = pd.read_csv(journals_ds_dir)
journals_df.head(1)

,Answer,Answer.f1.afraid.raw,Answer.f1.angry.raw,Answer.f1.anxious.raw,Answer.f1.ashamed.raw,Answer.f1.awkward.raw,Answer.f1.bored.raw,Answer.f1.calm.raw,Answer.f1.confused.raw,Answer.f1.disgusted.raw,...,Answer.t1.family.raw,Answer.t1.food.raw,Answer.t1.friends.raw,Answer.t1.god.raw,Answer.t1.health.raw,Answer.t1.love.raw,Answer.t1.recreation.raw,Answer.t1.school.raw,Answer.t1.sleep.raw,Answer.t1.work.raw
0,"My family was the most salient part of my day,...",False,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [8]:
journals = journals_df['Answer'].to_list()
len(journals), journals[:5]

(1473,
 ['My family was the most salient part of my day, since most days the care of my 2 children occupies the majority of my time. They are 2 years old and 7 months and I love them, but they also require so much attention that my anxiety is higher than ever. I am often overwhelmed by the care the require, but at the same, I am so excited to see them hit developmental and social milestones.',
  'Yoga keeps me focused. I am able to take some time for me and breath and work my body. This is important because it sets up my mood for the whole day.',
  'Yesterday, my family and I played a bunch of board games. My husband won most of them which is not surprising in the least. We played all sorts of games including Life, Clue, Mouse Trap and more. It was relaxing and such a happy, fun filled moment.',
  "Yesterday, I visited my parents and had dinner with them.  I hadn't seen them in a few weeks, so it was wonderful to see them and catch up on things.",
  'Yesterday, I really felt the import

In [9]:
from datasets import load_dataset


diaries_ds = load_dataset('chaudha7/Diary-Entry-To-Rap')
diaries_ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Age', 'Gender', 'Mood', 'Writing Manner', 'Diary Entry', 'Rap Song'],
        num_rows: 175
    })
})

In [10]:
diaries = diaries_ds['train']['Diary Entry']
len(diaries), diaries[:5]

(175,
 ['Dear Diary,\n\nI find solace in your pages today, as my weary soul seeks refuge from the relentless demands of the world. Fatigue has woven its heavy tendrils around my body, stifling my spirit and clouding my mind. The weariness seems never-ending, suffocating every ounce of energy that once resided within me.\n\nThe incessant noise and constant motion of life have suffused my being, leaving me utterly drained. The weight of expectations and responsibilities bears down upon my shoulders, making each step feel like a monumental effort. It feels as if I am eternally trapped in a labyrinth of tasks, forever searching for an escape that eludes me.\n\nIn this moment, all I desire is solitude; a chance to retreat into the sanctuary of my introverted nature. To find solace within myself and reconnect with the depths of my being. The mere thought of self-care becomes a distant dream, fading like a mirage in the scorching desert of my exhaustion.\n\nPerhaps tomorrow will bring respite

### Prepare for labeling

In [11]:
diary_texts = journals + diaries
len(diary_texts)

1648

### Labeling

In [22]:
from transformers import pipeline


reddit_classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None, padding=True, truncation=True)
twitter_classifier = pipeline(task="text-classification", model="cardiffnlp/twitter-roberta-base-emotion-multilabel-latest", top_k=None, padding=True, truncation=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClass

In [21]:
reddit_outputs = reddit_classifier(diary_texts)
reddit_outputs[:5]

[[{'label': 'love', 'score': 0.5109764933586121},
  {'label': 'excitement', 'score': 0.48795703053474426},
  {'label': 'nervousness', 'score': 0.08612798154354095},
  {'label': 'joy', 'score': 0.07194731384515762},
  {'label': 'fear', 'score': 0.050078731030225754},
  {'label': 'caring', 'score': 0.0349339060485363},
  {'label': 'approval', 'score': 0.033980388194322586},
  {'label': 'neutral', 'score': 0.02793497033417225},
  {'label': 'curiosity', 'score': 0.025223659351468086},
  {'label': 'desire', 'score': 0.022634319961071014},
  {'label': 'surprise', 'score': 0.019882917404174805},
  {'label': 'confusion', 'score': 0.019150804728269577},
  {'label': 'admiration', 'score': 0.018333662301301956},
  {'label': 'sadness', 'score': 0.01621655933558941},
  {'label': 'realization', 'score': 0.012149728834629059},
  {'label': 'optimism', 'score': 0.007913143374025822},
  {'label': 'disapproval', 'score': 0.00718197925016284},
  {'label': 'disappointment', 'score': 0.006426065694540739},


In [24]:
import pickle


with open('data/reddit_outputs.pickle', 'wb') as handle:
    pickle.dump(reddit_outputs, handle)
# with open('filename.pickle', 'rb') as handle:
    # b = pickle.load(handle)

In [25]:
twitter_outputs = twitter_classifier(diary_texts)
twitter_outputs[:5]

[[{'label': 'fear', 'score': 0.9501382112503052},
  {'label': 'joy', 'score': 0.8890569806098938},
  {'label': 'optimism', 'score': 0.5708929896354675},
  {'label': 'love', 'score': 0.47067156434059143},
  {'label': 'sadness', 'score': 0.16423656046390533},
  {'label': 'anticipation', 'score': 0.11126109957695007},
  {'label': 'trust', 'score': 0.10160717368125916},
  {'label': 'pessimism', 'score': 0.07006607949733734},
  {'label': 'surprise', 'score': 0.046277113258838654},
  {'label': 'disgust', 'score': 0.005908763501793146},
  {'label': 'anger', 'score': 0.0035218195989727974}],
 [{'label': 'optimism', 'score': 0.9848803281784058},
  {'label': 'joy', 'score': 0.9709051847457886},
  {'label': 'trust', 'score': 0.31404176354408264},
  {'label': 'anticipation', 'score': 0.17711025476455688},
  {'label': 'love', 'score': 0.10567871481180191},
  {'label': 'surprise', 'score': 0.038091741502285004},
  {'label': 'fear', 'score': 0.021933140233159065},
  {'label': 'sadness', 'score': 0.01

In [26]:
import pickle


with open('data/twitter_outputs.pickle', 'wb') as handle:
    pickle.dump(twitter_outputs, handle)
# with open('filename.pickle', 'rb') as handle:
    # b = pickle.load(handle)

In [41]:
data = []

for i, text in enumerate(diary_texts):
    labels = reddit_outputs[i]
    data.append({"Text": text, "Labels": labels})

df = pd.DataFrame(data)
df = pd.concat([df.drop(['Labels'], axis=1), df['Labels'].apply(lambda x: pd.Series({d['label']: d['score'] for d in x}))], axis=1)
df.to_csv('data/diaries_labeled_reddit.csv')
df.head()

/tmp/ipykernel_126710/31006852.py:8: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = pd.concat([df.drop(['Labels'], axis=1), df['Labels'].apply(lambda x: pd.Series({d['label']: d['score'] for d in x}))], axis=1)


,Text,love,excitement,nervousness,joy,fear,caring,approval,neutral,curiosity,...,relief,annoyance,anger,gratitude,pride,amusement,remorse,disgust,grief,embarrassment
0,"My family was the most salient part of my day,...",0.510976,0.487957,0.086128,0.071947,0.050079,0.034934,0.033980,0.027935,0.025224,...,0.005709,0.005669,0.004444,0.004387,0.003100,0.002959,0.002940,0.002605,0.002552,0.002357
1,Yoga keeps me focused. I am able to take some ...,0.002426,0.009715,0.003633,0.064329,0.001948,0.043616,0.443933,0.389298,0.000769,...,0.061554,0.004109,0.000569,0.006296,0.014715,0.001650,0.000956,0.000866,0.001463,0.000928
2,"Yesterday, my family and I played a bunch of b...",0.009383,0.046870,0.001557,0.888866,0.000708,0.009057,0.051839,0.036733,0.002599,...,0.019959,0.006183,0.002133,0.005249,0.005868,0.063577,0.000656,0.000640,0.000801,0.000949
3,"Yesterday, I visited my parents and had dinner...",0.069090,0.112845,0.002195,0.743193,0.000799,0.006355,0.042303,0.026994,0.002446,...,0.017489,0.004789,0.001385,0.008756,0.011549,0.005467,0.000875,0.000925,0.001729,0.001088
4,"Yesterday, I really felt the importance of my ...",0.005440,0.014871,0.006863,0.643160,0.002724,0.025474,0.110433,0.028650,0.000965,...,0.168369,0.008886,0.001731,0.059086,0.037131,0.004273,0.003204,0.001334,0.008456,0.002892


In [42]:
df.describe()

,love,excitement,nervousness,joy,fear,caring,approval,neutral,curiosity,desire,...,relief,annoyance,anger,gratitude,pride,amusement,remorse,disgust,grief,embarrassment
count,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,...,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000
mean,0.071604,0.059814,0.017506,0.334513,0.010016,0.036069,0.098302,0.106291,0.005599,0.027637,...,0.030764,0.042346,0.008475,0.025542,0.021390,0.015364,0.003901,0.004643,0.003061,0.004205
std,0.198698,0.136579,0.072084,0.332384,0.055784,0.092058,0.098079,0.163866,0.015741,0.087526,...,0.038996,0.118811,0.041106,0.096688,0.058975,0.060052,0.016468,0.034438,0.005077,0.030492
min,0.000352,0.000451,0.000125,0.000638,0.000252,0.000530,0.003862,0.003850,0.000317,0.000794,...,0.000291,0.001875,0.000398,0.000278,0.000233,0.000267,0.000183,0.000395,0.000257,0.000203
25%,0.002887,0.006766,0.001413,0.020071,0.000817,0.005261,0.031880,0.025316,0.001117,0.003734,...,0.004791,0.004484,0.001230,0.002761,0.002263,0.001851,0.000794,0.000809,0.001010,0.000826
50%,0.007437,0.018560,0.002534,0.186064,0.001312,0.012834,0.065716,0.044379,0.001708,0.005863,...,0.014577,0.006467,0.001904,0.008031,0.007330,0.003647,0.001365,0.001195,0.001575,0.001271
75%,0.021522,0.042637,0.005729,0.668107,0.002881,0.025325,0.123740,0.104639,0.002885,0.012483,...,0.041208,0.014129,0.003258,0.016810,0.015449,0.007641,0.003050,0.002441,0.002901,0.002954
max,0.963615,0.838949,0.637537,0.915659,0.827812,0.820288,0.682127,0.957725,0.234323,0.826884,...,0.249349,0.789217,0.779073,0.985614,0.459790,0.928085,0.514832,0.807316,0.079934,0.793776


In [37]:
data = []

for i, text in enumerate(diary_texts):
    labels = twitter_outputs[i]
    data.append({"Text": text, "Labels": labels})

df = pd.DataFrame(data)
df = pd.concat([df.drop(['Labels'], axis=1), df['Labels'].apply(lambda x: pd.Series({d['label']: d['score'] for d in x}))], axis=1)
df.to_csv('data/diaries_labeled_twitter.csv')
df.head()

/tmp/ipykernel_126710/3979847611.py:8: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = pd.concat([df.drop(['Labels'], axis=1), df['Labels'].apply(lambda x: pd.Series({d['label']: d['score'] for d in x}))], axis=1)


,Text,fear,joy,optimism,love,sadness,anticipation,trust,pessimism,surprise,disgust,anger
0,"My family was the most salient part of my day,...",0.950138,0.889057,0.570893,0.470672,0.164237,0.111261,0.101607,0.070066,0.046277,0.005909,0.003522
1,Yoga keeps me focused. I am able to take some ...,0.021933,0.970905,0.984880,0.105679,0.011022,0.177110,0.314042,0.008994,0.038092,0.006505,0.009273
2,"Yesterday, my family and I played a bunch of b...",0.015488,0.997395,0.875692,0.550680,0.013392,0.085346,0.098971,0.007222,0.076078,0.007966,0.007648
3,"Yesterday, I visited my parents and had dinner...",0.028493,0.997325,0.801768,0.890490,0.037286,0.105358,0.110605,0.012169,0.070891,0.007563,0.005539
4,"Yesterday, I really felt the importance of my ...",0.075375,0.962203,0.952603,0.213405,0.109883,0.044706,0.073201,0.014613,0.018295,0.005483,0.003692


In [40]:
df.describe()

,fear,joy,optimism,love,sadness,anticipation,trust,pessimism,surprise,disgust,anger
count,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000,1648.000000
mean,0.129375,0.765271,0.704569,0.318703,0.209978,0.170029,0.120912,0.105010,0.055638,0.097426,0.086413
std,0.232147,0.387562,0.343908,0.328517,0.346009,0.163020,0.107103,0.199005,0.057314,0.237001,0.236331
min,0.003498,0.001681,0.002834,0.001574,0.003670,0.011112,0.002713,0.004637,0.003730,0.002939,0.001720
25%,0.022724,0.775871,0.642946,0.027351,0.012137,0.064381,0.038925,0.008696,0.029654,0.006749,0.006412
50%,0.034819,0.990664,0.873755,0.172366,0.021323,0.110228,0.100913,0.012382,0.048168,0.009055,0.008813
75%,0.085634,0.995546,0.939659,0.594024,0.188202,0.213210,0.162594,0.049038,0.066001,0.018581,0.015027
max,0.994671,0.997845,0.996070,0.977586,0.994248,0.937603,0.635685,0.841152,0.913347,0.985776,0.993624


## 2. Reddit and Twitter Posts Datasets

Datasets:

1. [go_emotions](https://huggingface.co/datasets/go_emotions) — 58k carefully curated Reddit comments labeled for 27 emotion categories or Neutral.
2. [sem_eval_2018_task_1](https://huggingface.co/datasets/sem_eval_2018_task_1) — a dataset of Twitter tweets in Arabic, English (11k entries), Spanish to automatically determine the intensity of emotions (E) and intensity of sentiment (aka valence V) of the tweeters from their tweets.

Both datasets are designed to solve multi-label emotion classification task. From both datasets only texts will be taken.

TODO: try